In [1]:
# Show all Jupyter output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
# import files from above
import sys
sys.path.append('..')

In [2]:



import torch
import torch.utils.data as data
from ignite.engine import Engine

from src.datasets import WarpDataset
from src.loss import PerPixelCrossEntropyLoss
from src.nets import Discriminator, GeneratorUNet
from src.warp_module import WarpModule

from config import *

In [3]:
cuda = True if torch.cuda.is_available() else False
cuda

True

In [4]:
clothing_dir = "test_resources/clothing_seg"
body_dir = "test_resources/body_seg"

In [5]:
def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find("BatchNorm2d") != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)

In [6]:
dataset = WarpDataset(
    clothing_seg_dir=clothing_dir, 
    body_seg_dir=body_dir, 
    body_means=ANDREW_BODY_SEG_MEAN, 
    body_stds=ANDREW_BODY_SEG_STD
)
data_loader = torch.utils.data.DataLoader(dataset, batch_size=2)

In [7]:
# Loss Warp function
reconstruction_loss = PerPixelCrossEntropyLoss()
adversarial_loss = torch.nn.BCELoss()
# later: warp_loss = reconstruction_loss(i, t) + adversarial_loss(d, c)
# warp_loss.backward()
generator = WarpModule()
discriminator = Discriminator()
if cuda:
    reconstruction_loss = reconstruction_loss.cuda()
    adversarial_loss = adversarial_loss.cuda()
    generator = generator.cuda()
    discriminator = discriminator.cuda()

next(generator.parameters()).is_cuda

True

In [8]:
generator.apply(weights_init_normal)
discriminator.apply(weights_init_normal)

WarpModule(
  (body_down1): UNetDown(
    (model): Sequential(
      (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): LeakyReLU(negative_slope=0.2)
    )
  )
  (body_down2): UNetDown(
    (model): Sequential(
      (0): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (2): LeakyReLU(negative_slope=0.2)
    )
  )
  (body_down3): UNetDown(
    (model): Sequential(
      (0): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (2): LeakyReLU(negative_slope=0.2)
    )
  )
  (body_down4): UNetDown(
    (model): Sequential(
      (0): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): InstanceNorm2d(512, eps=1e-05, momentum=0.1, affine=False, track_r

Discriminator(
  (model): Sequential(
    (0): Conv2d(6, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2, inplace)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (3): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (4): LeakyReLU(negative_slope=0.2, inplace)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (6): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (7): LeakyReLU(negative_slope=0.2, inplace)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (9): InstanceNorm2d(512, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (10): LeakyReLU(negative_slope=0.2, inplace)
    (11): ZeroPad2d(padding=(1, 0, 1, 0), value=0)
    (12): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1), bias=False)
  )
)

In [9]:
optimizer_G = torch.optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.99))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.99))

In [10]:
body, input, target = next(iter(data_loader))

In [11]:
body = body.cuda()
body.shape
input.shape
body.is_cuda

torch.Size([2, 3, 960, 540])

torch.Size([2, 19, 960, 540])

True

In [12]:
body.max()
body.min()
body.mean()

tensor(4.3224, device='cuda:0')

tensor(-0.2993, device='cuda:0')

tensor(-0.0445, device='cuda:0')

In [13]:
cropped_body = body[:, :, :512, :512]
cropped_body.shape
cropped_body.is_cuda

torch.Size([2, 3, 512, 512])

True

In [14]:
input.max()
input.min()
input.mean()

tensor(0.9994)

tensor(2.1741e-08)

tensor(0.0526)

In [15]:
input = input.cuda()
input.is_cuda

cropped_input = input[:, :, :512, :512]
cropped_input.shape
cropped_input.is_cuda

True

torch.Size([2, 19, 512, 512])

True

In [16]:
x = generator(cropped_body, cropped_input)

/home/espina/.conda/envs/outfit/lib/python3.7/site-packages/torch/nn/modules/upsampling.py:122: UserWarning: nn.Upsampling is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.Upsampling is deprecated. Use nn.functional.interpolate instead.")


In [19]:
x.shape
x.mean()

torch.Size([2, 19, 512, 512])

tensor(-0.0193, device='cuda:0', grad_fn=<MeanBackward1>)

### Test out that loss works with the generator output

In [20]:
from src.loss import PerPixelCrossEntropyLoss

In [21]:
loss = PerPixelCrossEntropyLoss().cuda()

In [33]:
cropped_target = target.cuda()[[0],:,:512, :512]
cropped_target.shape


first_output = x[[0],:,:512, :512]
first_output.shape

torch.Size([1, 19, 512, 512])

torch.Size([1, 19, 512, 512])

In [35]:
loss(first_output, cropped_target)

tensor(2.9648, device='cuda:0', grad_fn=<NllLoss2DBackward>)